# Mixed-member-proportional election using the New Zealand electoral system

### Using the New Zealand system as inspiration, simulate the results of the election if Canada used the same system
https://en.wikipedia.org/wiki/Electoral_system_of_New_Zealand

## Setup

In [2]:
# Imports
import math
import numpy as np
import pandas as pd

In [3]:
# Import data

election_data = pd.read_csv('../data/2019-canada-federal-results.csv')

In [4]:
# Generate metadata

ridings = {r['Riding ID'] for i, r in election_data.iterrows()}
provinces = {r['Province ID'] for i, r in election_data.iterrows()}
parties = {r['Party ID'] for i, r in election_data.iterrows()}

In [5]:
# Validate the votes total
# 2019 Federal Total Valid Votes = 18,170,880 (from elections.ca)

election_data['Votes'].sum()

18170880

## Convert FPTP data to MMP data

In [6]:
# In MMP, each person gets two votes, a party vote and a local representative vote

election_data['Party Votes'] = election_data['Votes']

# Remove independent party votes
election_data.loc[election_data['Party ID'] == 'IND', 'Party Votes'] = 0

# Rename votes to local votes
election_data = election_data.rename(columns={'Votes': 'Local Votes'})

## Declare winners in each riding

In [7]:
# Add a winner column to the dataframe

election_data['Winner'] = False

In [8]:
# For each riding, set the winner

for riding in ridings:
    # Grab the relevant rows for the riding
    riding_data = election_data.loc[election_data['Riding ID'] == riding]
    
    # Determine the index of the candidate with the maximum votes
    winner_index = riding_data['Local Votes'].idxmax()
    
    # Set the winner column to True for that candidate
    election_data.loc[winner_index, 'Winner'] = True

In [9]:
# Generate a table of just the winners

winners = election_data.loc[election_data['Winner'] == True]

In [10]:
# Verify we selected 338 winners

len(winners.index)

338

## Determine the total seat distribution

In [11]:
# Determine which parties are eligible for list seat distribution
# In the New Zealand system, parties are eligible if they receive at least 5% of the party vote, or if they win a riding

threshold = 0.05

eligible_parties = []
for party in parties:
    # Add them to eligible if they received >5% of the total vote
    if election_data.loc[election_data['Party ID'] == party, 'Party Votes'].sum() >= election_data['Party Votes'].sum() * threshold:
        eligible_parties.append(party)
        continue
    
    # Add them to eligible if they won at least one riding
    if len(winners.loc[winners['Party ID'] == party].index) > 0:
        eligible_parties.append(party)

# Remove independents
if 'IND' in eligible_parties:
    eligible_parties.remove('IND')

eligible_parties

['BQ', 'LPC', 'GRE', 'NDP', 'CPC']

In [12]:
# Determine how many list seats there will be
# In New Zealand, the split between local seats and list seats is 60/40

local_seats = 338
proportion_local = 0.6

list_seats = local_seats / proportion_local - local_seats
list_seats = math.floor(list_seats)  # Flooring to remove fractional seats

total_seats = local_seats + list_seats

print('Local seats:', local_seats, 'List seats:', list_seats, 'Total seats:', total_seats)

Local seats: 338 List seats: 225 Total seats: 563


In [13]:
# Calculate the number of seats to allocate
# Normally, this would just be the total number of seats in parliament, but if an independent wins a riding, the seats to allocate is reduced

seats_to_allocate = total_seats - len(winners.loc[winners['Party ID'] == 'IND'].index)
seats_to_allocate

562

In [14]:
# Build the quotient table, using the Sainte-Lague method to determint the quotients for each party
# The highest x quotients are selected for seats, where x is the number of seats to allocate

columns = ['Divisor'] + eligible_parties
quotient_table = pd.DataFrame(columns=columns, index=range(1, seats_to_allocate + 1))

# Calculate the divisors
quotient_table['Divisor'] = 2 * quotient_table.index - 1

# Fill in each party's column
for party in eligible_parties:
    quotient_table[party] = election_data.loc[election_data['Party ID'] == party, 'Party Votes'].sum()
    quotient_table[party] = quotient_table[party] / quotient_table['Divisor']

quotient_table.head().style.format('{:.1f}')

,Divisor,BQ,LPC,GRE,NDP,CPC
1,1.0,1387030.0,6018728.0,1189607.0,2903722.0,6239227.0
2,3.0,462343.3,2006242.7,396535.7,967907.3,2079742.3
3,5.0,277406.0,1203745.6,237921.4,580744.4,1247845.4
4,7.0,198147.1,859818.3,169943.9,414817.4,891318.1
5,9.0,154114.4,668747.6,132178.6,322635.8,693247.4


In [15]:
# Determine the x highest quotients from all parties

# First, crunch all the quotients into a list
quotients = []

for party in eligible_parties:
    quotients += quotient_table[party].to_list()

quotients.sort(reverse=True)

# Select the x largest, where x is the seats to allocate 
# Meaning that the lowest quotient to be elected is the xth quotient
quotient_required = quotients[:seats_to_allocate][-1]
quotient_required

15797.186351706037

In [16]:
# Determine how many seats each party is entitled to

total_seat_allocation = {}
for party in eligible_parties:
    total_seat_allocation[party] = len(quotient_table.loc[quotient_table[party] >= quotient_required].index)

total_seat_allocation

{'BQ': 44, 'LPC': 191, 'GRE': 38, 'NDP': 92, 'CPC': 197}

## Summarize the results by party

In [17]:
# Create a table of the results by party

party_results = pd.DataFrame(columns=['Local Seats', 'Local Seats %', 'List Seats', 'List Seats %', 'Total Seats', 'Total Seats %', 'Local Votes', 'Local Votes %', 'Party Votes', 'Party Votes %'], index=list(parties))

# Calculate results by party
for party in parties:
    # Determine how many local seats were won
    party_results.loc[party, 'Local Seats'] = len(winners.loc[winners['Party ID'] == party].index)
    # Determine how many local and party votes were received
    party_results.loc[party, 'Local Votes'] = election_data.loc[election_data['Party ID'] == party, 'Local Votes'].sum()
    party_results.loc[party, 'Party Votes'] = election_data.loc[election_data['Party ID'] == party, 'Party Votes'].sum()

In [18]:
# Determine how many list seats to give each party based on the total allocation

for party in parties:
    # If the party has received an allocation, use that, otherwise set to zero
    if party in total_seat_allocation.keys():
        list_seats = total_seat_allocation[party] - party_results.loc[party, 'Local Seats']
    else:
        list_seats = 0

    # If list seats is negative, an overhang has occured
    if list_seats < 0:
        list_seats = 0
    
    party_results.loc[party, 'List Seats'] = list_seats

# Calculate the total seats
party_results['Total Seats'] = party_results['Local Seats'] + party_results['List Seats']

In [19]:
# Fill in % columns

party_results['Local Seats %'] = party_results['Local Seats'] / party_results['Local Seats'].sum() * 100
party_results['List Seats %'] = party_results['List Seats'] / party_results['List Seats'].sum() * 100
party_results['Total Seats %'] = party_results['Total Seats'] / party_results['Total Seats'].sum() * 100
party_results['Local Votes %'] = party_results['Local Votes'] / party_results['Local Votes'].sum() * 100
party_results['Party Votes %'] = party_results['Party Votes'] / party_results['Party Votes'].sum() * 100

In [20]:
# Sort and validate the results
# 2019 Federal = LPC 157 | CPC 121 | BQ 32 | NDP 24 | GRE 3 | IND 1

party_results.loc[(party_results['Party Votes %'] > 1) | (party_results['Total Seats'] > 0)].sort_values(by=['Total Seats', 'Party Votes'], ascending=False).style.format('{:.1f}')

,Local Seats,Local Seats %,List Seats,List Seats %,Total Seats,Total Seats %,Local Votes,Local Votes %,Party Votes,Party Votes %
CPC,121.0,35.8,76.0,33.8,197.0,35.0,6239227.0,34.3,6239227.0,34.5
LPC,157.0,46.4,34.0,15.1,191.0,33.9,6018728.0,33.1,6018728.0,33.3
NDP,24.0,7.1,68.0,30.2,92.0,16.3,2903722.0,16.0,2903722.0,16.0
BQ,32.0,9.5,12.0,5.3,44.0,7.8,1387030.0,7.6,1387030.0,7.7
GRE,3.0,0.9,35.0,15.6,38.0,6.7,1189607.0,6.5,1189607.0,6.6
IND,1.0,0.3,0.0,0.0,1.0,0.2,74291.0,0.4,0.0,0.0
PPC,0.0,0.0,0.0,0.0,0.0,0.0,294092.0,1.6,294092.0,1.6


## Calculate the Gallagher Index
Measure of election disproprtionality between seats received and votes received. (https://en.wikipedia.org/wiki/Gallagher_index)

In [21]:
# Add to the table to determine the Gallagher index (sqrt of half the sum of the squared difference between seats and votes)

party_results['Diff'] = party_results['Total Seats %'] - party_results['Party Votes %']
party_results['Diff**2'] = party_results['Diff'] ** 2

In [22]:
# Calculate the Gallagher index

gallagher_index = math.sqrt(party_results['Diff**2'].sum() / 2)
print('Gallagher index is', round(gallagher_index, 2))

Gallagher index is 1.33


## Export the results

In [23]:
# Start an ExcelWriter to export the results

with pd.ExcelWriter('results.xlsx') as xls:
    party_results.sort_values(by=['Total Seats', 'Party Votes'], ascending=False).to_excel(xls, sheet_name='Party Results')
    election_data.to_excel(xls, sheet_name='Details')